# Data Processing

## 1. Global Settings

순서 파악파기

python parse_behavior.py --in-file "$datasetTrainPath/behaviors.tsv" --out-dir "$preTrainPath" --mode train

python parse_behavior.py --in-file "$datasetTestPath/behaviors.tsv" --out-dir "$preTestPath" --mode test --user2int "$preTrainPath/user2int.tsv"

python parse_news.py --in-file "$datasetTrainPath/news.tsv" --out-dir "$preTrainPath" --mode train --word-embeddings "$wordEmbeddingPath/glove.840B.300d.txt"

python parse_news.py --in-file "$datasetTestPath/news.tsv" --out-dir "$preTestPath" --mode test --word-embeddings "$wordEmbeddingPath/glove.840B.300d.txt" --embedding-weights "$preTrainPath/embedding_weights.csv" --word2int "$preTrainPath/word2int.tsv" --category2int "$preTrainPath/category2int.tsv"

parse_behavior.py 먼저 진행

parse_news는 glove 임베딩 파일 필요

In [ ]:
from sklearn.model_selection import train_test_split

import argparse
import parse_behavior
import parse_news

In [ ]:
size = "demo"

datasetPath = f"MIND/{size}"
datasetTrainPath = f"{datasetPath}/train"
datasetTestPath = f"{datasetPath}/test"
wordEmbeddingPath = "word_embeddings"

processedDataPath = "processed"
preTrainPath = f"{processedDataPath}/{size}/train"
preTestPath = f"{processedDataPath}/{size}/test"

user2int = f"{preTrainPath}/user2int.tsv"

word_embeddings = f"{wordEmbeddingPath}/glove.840B.300d.txt"
word2int = f"{preTrainPath}/word2int.tsv"

In [ ]:
args = argparse.Namespace(
    config = "config/model/nrms/exp1.yaml",
    mode = "train"
)

In [ ]:
# train/test behaviors.tsv 전처리

if(args.mode == "train"):
    with open(args.in_file, 'r') as in_file:
        behavior = in_file.readlines()
        if (args.split == 0):
            parse_behavior.generate_training_data(behavior, args.out_dir)
        else:
            train_behavior, val_behavior = train_test_split(behavior,test_size=args.split, random_state=1234)
            user2int = parse_behavior.generate_training_data(train_behavior, args.out_dir)
            parse_behavior.generate_eval_data(val_behavior, args.out_dir, "val_behavior.tsv", user2int)
elif(args.mode == "test"):
    user2int = parse_behavior.load_idx_map_as_dict(args.user2int)
    with open(args.in_file, 'r') as in_file:
        behavior = in_file.readlines()
        parse_behavior.generate_eval_data(behavior, args.out_dir, "test_behavior.tsv", user2int)
else:
    print("Wrong mode!")